# Sentinel-1 EW RTC with pyroSAR (GAMMA)

## Imports

In [ ]:
# initial setup
import os
import asf_search as asf
from eof.download import download_eofs
import yaml
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import shutil
from shapely import Polygon, box
import numpy as np
import pyproj
import rasterio
import rasterio.merge
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyroSAR.snap import geocode

## Settings

In [ ]:
GAMMA_HOME_PATH = "/g/data/dg9/GAMMA/GAMMA_SOFTWARE-20230712"

if os.environ.get("GAMMA_HOME", None) is not None:
    os.environ["GAMMA_HOME"] = GAMMA_HOME_PATH

### Confirm PyroSAR can access GAMMA install

In [ ]:
from pyroSAR.examine import ExamineGamma
config = ExamineGamma()

print(config.home)
print(config.version)

### Paths

In [ ]:
# paths
data_dir =  '/g/data/yp75/ca6983/data'
download_folder = os.path.join(data_dir,'scenes')
earthdata_credentials_path = "credentials/credentials_earthdata.yaml"
copernicus_credentials_path = "credentials/credentials_copernicus.yaml"
precise_orb_download_folder = os.path.join(data_dir,'orbits','POEORB')
restituted_orb_download_folder = os.path.join(data_dir,'orbits','RESORB')
rtc_outpath = os.path.join(data_dir,'results')
rtc_scratch_dir = os.path.join(data_dir,'scratch')
dem_download_folder = os.path.join(data_dir,'dem')

In [ ]:
create_folders = True
if create_folders:
    for f in [
        download_folder, 
        precise_orb_download_folder, 
        restituted_orb_download_folder,
        dem_download_folder,
        rtc_outpath,
        rtc_scratch_dir
        ]:
        os.makedirs(f, exist_ok=True)

### Credentials

In [ ]:
with open(earthdata_credentials_path, "r", encoding='utf8') as f:
        earthdata_credentials = yaml.safe_load(f.read())
with open(copernicus_credentials_path, "r", encoding='utf8') as f:
        copernicus_credentials = yaml.safe_load(f.read())

## Set up data

In [ ]:
#s1_zip_file = "/g/data/fj7/Copernicus/Sentinel-1/C-SAR/GRD/2024/2024-01/70S170E-75S175E/S1A_EW_GRDM_1SDH_20240129T091735_20240129T091828_052319_065379_0F1E.zip"

scene_id = "S1A_EW_GRDM_1SDH_20240129T091735_20240129T091828_052319_065379_0F1E"
prod = "GRD_MD"
mode = "EW"

results = asf.granule_search([scene_id], asf.ASFSearchOptions(processingLevel=prod, beamMode=mode))

In [ ]:
session = asf.ASFSession()
session.auth_with_creds(earthdata_credentials['login'],earthdata_credentials['password'])

In [ ]:
# limit to the first scene
results = results[0:1]
geometry = results[0].geometry
scene = results[0].properties['sceneName']
pol = results[0].properties['polarization']
name = str(results[0].properties['sceneName'])

In [ ]:
# download all results
scene_paths = []
scene_names = []
for s in results:
    name = s.properties['sceneName']
    scene_names.append(name)
    print(name)
    path = os.path.join(download_folder, name)
    s.download(path=download_folder, session=session)
    scene_paths.append(path)

In [ ]:
scene_paths

In [ ]:
from pyroSAR import identify

id = identify(scene_paths[0] + ".zip")

In [ ]:
def transform_polygon(src_crs, dst_crs, geometry, always_xy=True):
    src_crs = pyproj.CRS(f"EPSG:{src_crs}")
    dst_crs = pyproj.CRS(f"EPSG:{dst_crs}") 
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=always_xy)
     # Transform the polygon's coordinates
    transformed_exterior = [
        transformer.transform(x, y) for x, y in geometry.exterior.coords
    ]
    # Create a new Shapely polygon with the transformed coordinates
    transformed_polygon = Polygon(transformed_exterior)
    return transformed_polygon

def adjust_scene_poly_at_extreme_lat(bbox, src_crs, ref_crs, delta=0.1):
    """
    Adjust the bounding box around a scene in src_crs (4326) due to warping at high
    Latitudes. For example, the min and max bounding values for an antarctic scene in
    4326 may not actually be the true min and max due to distortions at high latitudes. 

    Parameters:
    - bbox: Tuple of four coordinates (x_min, y_min, x_max, y_max).
    - src_crs: Source EPSG. e.g. 4326
    - ref_crs: reference crs to create the true bbox. i.e. 3031 in southern 
                hemisphere and 3995 in northern (polar stereographic)
    - delta: distance between generation points along the bounding box sides in
            src_crs. e.g. 0.1 degrees in lat/lon 

    Returns:
    - A polygon bounding box expanded to the true min max
    """
    x_min, y_min, x_max, y_max = bbox
    # Generate points along the top side
    top_side = [(x, y_max) for x in list(np.arange(x_min, x_max, delta)) + [x_max]]    
    # Generate points along the right side
    right_side = [(x_max, y) for y in list(np.arange(y_max - delta, y_min-delta, -delta)) + [y_min]]
    # Generate points along the bottom side
    bottom_side = [(x, y_min) for x in list(np.arange(x_max - delta, x_min-delta, -delta)) + [x_min]]
    list(np.arange(y_min + delta, y_max, delta)) + [y_max]
    # Generate points along the left side
    left_side = [(x_min, y) for y in list(np.arange(y_min + delta, y_max, delta)) + [y_max]]
    # Combine all sides' points
    all_points = top_side + right_side + bottom_side + left_side
    # convert to a polygon 
    polygon = Polygon(all_points)
    # convert polygon to desired crs and get bounds in those coordinates
    trans_bounds = transform_polygon(src_crs, ref_crs, polygon).bounds
    trans_poly = Polygon(
        [(trans_bounds[0], trans_bounds[1]), 
         (trans_bounds[2], trans_bounds[1]), 
         (trans_bounds[2], trans_bounds[3]), 
         (trans_bounds[0], trans_bounds[3])]
        )
    corrected_poly = transform_polygon(ref_crs, src_crs, trans_poly)
    return corrected_poly

def expand_raster_with_bounds(input_raster, output_raster, old_bounds, new_bounds, fill_value=None):
    """Expand the raster to the desired bounds. Resolution and Location are preserved.

    Args:
        input_raster (str): input raster path
        output_raster (str): out raster path
        old_bounds (tuple): current bounds
        new_bounds (tuple): new bounds
        fill_value (float, int, optional): Fill value to pad with. Defaults to None and nodata is used.
    """
    # Open the raster dataset
    with rasterio.open(input_raster, 'r') as src:
        # get old bounds
        old_left, old_bottom, old_right, old_top = old_bounds
        # Define the new bounds
        new_left, new_bottom, new_right, new_top = new_bounds
        # adjust the new bounds with even pixel multiples of existing
        # this will stop small offsets
        print(f'Making new raster with target bounds: {new_bounds}')
        new_left = old_left - int(abs(new_left-old_left)/src.res[0])*src.res[0]
        new_right = old_right + int(abs(new_right-old_right)/src.res[0])*src.res[0]
        new_bottom = old_bottom - int(abs(new_bottom-old_bottom)/src.res[1])*src.res[1]
        new_top = old_top + int(abs(new_top-old_top)/src.res[1])*src.res[1]
        print(f'New raster bounds: {(new_left, new_bottom, new_right, new_top)}')
        # Calculate the new width and height, should be integer values
        new_width = int((new_right - new_left) / src.res[0])
        new_height = int((new_top - new_bottom) / src.res[1])
        # Define the new transformation matrix
        transform = from_origin(new_left, new_top, src.res[0], src.res[1])
        # Create a new raster dataset with expanded bounds
        profile = src.profile
        profile.update({
            'width': new_width,
            'height': new_height,
            'transform': transform
        })
        # make a temp file
        tmp = output_raster.replace('.tif','_tmp.tif')
        print(f'Making temp file: {tmp}')
        with rasterio.open(tmp, 'w', **profile) as dst:
            # Read the data from the source and write it to the destination
            fill_value = profile['nodata'] if fill_value is None else fill_value
            print(f'Padding new raster extent with value: {fill_value}')
            data = np.full((new_height, new_width), fill_value=fill_value, dtype=profile['dtype'])
            dst.write(data, 1)
        # merge the old raster into the new raster with expanded bounds 
        print(f'Merging original raster and expanding bounds...')
    del data
    rasterio.merge.merge(
        datasets=[tmp, input_raster],
        method='max',
        dst_path=output_raster)
    os.remove(tmp)

In [ ]:
# buffer the sceme nounda to ensure coverage
scene_bounds = Polygon(geometry['coordinates'][0]).bounds
print(f'original scene bounds : {scene_bounds}')

In [ ]:
# if we are at high latitudes we need to correct the bounds due to the skewed box shape
if (scene_bounds[1] < -50) or (scene_bounds[3] < -50):
    # Southern Hemisphere
    print(f'Adjusting scene bounds due to warping at high latitude')
    scene_poly = adjust_scene_poly_at_extreme_lat(scene_bounds, 4326, 3031)
    scene_bounds = scene_poly.bounds 
    print(f'Adjusted scene bounds : {scene_bounds}')
if (scene_bounds[1] > 50) or (scene_bounds[3] > 50):
    # Northern Hemisphere
    print(f'Adjusting scene bounds due to warping at high latitude')
    scene_poly = adjust_scene_poly_at_extreme_lat(scene_bounds, 4326, 3995)
    scene_bounds = scene_poly.bounds 
    print(f'Adjusted scene bounds : {scene_bounds}')
print('adjusted scene bounds')
scene_bounds
# Create the polygon
geom = Polygon([(scene_bounds[0], scene_bounds[1]), 
                (scene_bounds[2], scene_bounds[1]), 
                (scene_bounds[2], scene_bounds[3]), 
                (scene_bounds[0], scene_bounds[3])])
geom = list(geom.exterior.coords)

In [ ]:
min_lat, max_lat  = min([c[1] for c in geom]), max([c[1] for c in geom])
min_lon, max_lon  = min([c[0] for c in geom]), max([c[0] for c in geom])
lats = list(range(int(np.floor(min_lat)), int(np.ceil(max_lat+1))))
longs = list(range(int(np.floor(min_lon)), int(np.ceil(max_lon+1))))
print(geom)
print(lats)
print(longs)

In [ ]:
dem_s3_paths = []

for lat in lats:
    for long in longs:
        if lat >= 0:
            lat_dir = "N"
        else:
            lat_dir = "S"
        if long >= 0:
            long_dir = "E"
        else:
            long_dir = "W"
        
        dem_s3_paths.append(f"Copernicus_DSM_COG_10_{lat_dir}{int(abs(lat)):02d}_00_{long_dir}{int(abs(long)):03d}_00_DEM/Copernicus_DSM_COG_10_{lat_dir}{int(abs(lat)):02d}_00_{long_dir}{int(abs(long)):03d}_00_DEM.tif")
dem_s3_paths = list(set(dem_s3_paths))
print(f'DEM tiles to download : {len(dem_s3_paths)}')

In [ ]:
s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
bucket_name = "copernicus-dem-30m"
bucket = s3.Bucket(bucket_name)
dems_to_merge = []

for s3_path in dem_s3_paths:
    try:
        dl_path = os.path.join(dem_download_folder,s3_path.split('/')[1])
        if not os.path.exists(dl_path):
            bucket.download_file(s3_path, dl_path)
            print(f'downloaded : {s3_path}')
            dems_to_merge.append(dl_path)
        else:
            print(f'file exists : {s3_path}')
            dems_to_merge.append(dl_path)
    except:
        print(f'not found : {s3_path}')

In [ ]:
dem_paths = ' '.join(dems_to_merge)
dem_paths

In [ ]:
merged_dem_path = os.path.join(dem_download_folder,f"{name}_dem.tif")

In [ ]:
print(f"gdal_merge -n 0 -o {merged_dem_path} {dem_paths}")

In [ ]:
with rasterio.open(merged_dem_path) as src:
    dem_meta = src.meta.copy()

In [ ]:
dem_meta